In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import iblofunmatch.inter as ibfm
import os

from navground import core, sim

import perdiver.perdiver as perdiver
from perdiver.distances import *

plots_dir = "plots/"
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

In [ ]:
num_steps = 2000
side=5.0
# num_agents = 28
num_agents = 20
# num_steps = 0
shift_time = 100
# shift_time = 50
steps_list = list(range(300, 330, 1))
diags_list = []
weight= 3
yaml = f"""
steps: {num_steps}
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Cross
  side: {side}
  groups:
    -
      type: thymio
      number: {num_agents}
      radius: 0.08
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: 0.166
      behavior:
        type: ORCA
        optimal_speed: 0.12
        horizon: 5.0
        safety_margin: 0.034
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds
# COmpute persistence diagrams and store
diags = []
for idx, start_step in enumerate(steps_list):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_corridor(ps, twists, start_step, shift_time, weight, length)
    diags.append(perdiver.get_matching_diagram(Dist_X, Dist_Y, Dist_Z, output_dir))

In [ ]:
len(diags)

Compute persistence matching diagrams

In [ ]:
from gudhi import representations
npixels = 30
xmax = np.max(diags)*1.1
ymax = xmax

Plot point cloud at two timesteps

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
shift_time = 100
color_list = ["blue", "red"]
for i in range(2):
    start_step = steps_list[i*10]
    perdiver.plot_two_timesteps(ps[start_step], ps[start_step + shift_time], ax[0], X_col=color_list[i], Y_col=color_list[i])
    # Compute matching 
    Dist_X = distances_weighted_velocities(ps[start_step], twists[start_step], weight)
    Dist_Y = distances_weighted_velocities(ps[start_step + shift_time], twists[start_step + shift_time], weight)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    matching_diag = perdiver.get_matching_diagram(Dist_X, Dist_Y, Dist_Z, output_dir)
    perdiver.plot_matching_diagram(matching_diag, ax[1], color=color_list[i])

plt.tight_layout()
plt.savefig(f"{plots_dir}stability_diagram.png")
# plt.savefig(f"{plots_dir}matching_diagram.png")
# plt.savefig(f"{plots_dir}matching_diagram_next.png")

In [ ]:
for i, (image, diag) in enumerate(zip(ORCA_perimages, diags_list[0])):
    fig, ax = plt.subplots(ncols=2, figsize=(8,4))
    image = image.reshape(npixels,-1)
    ax[0].imshow(image)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    perdiver.plot_matching_diagram(diag, ax[1], max_val_diag=xmax)
    fig.suptitle(f"{behaviour_list[0]} step: {steps_list[i]:5d}")
    plt.tight_layout()
    plt.savefig(plots_dir + f"ORCA_{steps_list[i]:04d}" + ".png")
    plt.close()

In [ ]:
for i, (image, diag) in enumerate(zip(HL_perimages, diags_list[1])):
    fig, ax = plt.subplots(ncols=2, figsize=(8,4))
    image = image.reshape(npixels,-1)
    ax[0].imshow(image)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    perdiver.plot_matching_diagram(diag, ax[1], max_val_diag=xmax)
    fig.suptitle(f"{behaviour_list[1]} step: {steps_list[i]:5d}")
    plt.tight_layout()
    plt.savefig(plots_dir + f"HL_{steps_list[i]:04d}" + ".png")
    plt.close()

# Plot videos of both experiments to see how they behave

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx
from navground.sim.ui.video import record_video

def linear_map(a, b, cmap):
    c = cmx.ScalarMappable(norm=colors.Normalize(vmin=a, vmax=b), cmap=cmap)  
    def f(v):
        r, g, b, _ = c.to_rgba(v)
        return f"#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}"
    return f


fill_map = linear_map(0.0, 1.0, cmap=cmx.RdYlGn)

def f(entity):
    if isinstance(entity, sim.Agent):
        return {'fill': fill_map(entity.behavior.efficacy)}
    return {}
    

In [ ]:
for behaviour in ["ORCA", "HL"]:
    yaml = f"""
    steps: 2
    time_step: 0.1
    save_directory: ''
    record_pose: true
    record_twist: true
    scenario:
      type: Corridor
      length: {length}
      width: {width} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 0.12
            horizon: 5.0
            safety_margin: 0.034
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run()
    run = experiment.runs[0]
    # Record video
    record_video(f"{plots_dir}{behaviour}_exp.mp4", run.world, time_step=0.1, duration=200.0, factor=14.0,
                  bounds=((0, 0), (length, width)), decorate=f, width=700, display_shape=True)

Now, we can see the generated videos

In [ ]:
from IPython.display import Video

Video(f"{plots_dir}ORCA_exp.mp4", width=700)

In [ ]:
from IPython.display import Video

Video(f"{plots_dir}HL_exp.mp4", width=700)